In [ ]:
import pandas as pd
import numpy as np
import string as s
import seaborn as sns
import math
# import tables
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import style, colors
from itertools import groupby
import matplotlib.patches as patches
from matplotlib import gridspec
import scipy.stats as stats
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster, fclusterdata, leaves_list
from scipy.spatial.distance import pdist
from scipy.ndimage.interpolation import shift
# matplotlib.style.use('ggplot')
# %matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12, 6)
# matplotlib.rcParams['figure.facecolor'] = 'white'
import mpl_toolkits.mplot3d.axes3d as p3
from sklearn.cluster import AgglomerativeClustering
from sklearn.datasets.samples_generator import make_swiss_roll
matplotlib.style.use('ggplot')
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# constants

In [ ]:
wkdir = '/projects/da_workspace/software/ChromHMM/cervical_all_patients_new_h3k36me3'
mark = ["H3K27ac","H3K27me3","H3K36me3","H3K4me1","H3K4me3","H3K9me3"]

# running ChromHMM

# prep input files: cellmarkerfiletable.txt, symlink peak files in  peak_bed_files directory

In [7]:
%%bash -s $wkdir $mark

wkdir=$1
# mark=$2
echo $wkdir, $mark
cd $wkdir
for mark in H3K27ac H3K27me3 H3K36me3 H3K4me1 H3K4me3 H3K9me3
do
    ls -1 /projects/chipseq/macs/A*/75nt/hg19a/bwa-mem-0.7.6a-sb/MACS_2.1.1.20160309/*/*$mark*Peak|grep -v gappedPeak > a.tmp
    cat a.tmp |awk -F "/" '{print $5"\t"$0}'|awk -F "_" '{print $3"\t"$0}'>d.tmp

    while read a b c;do grep $b 57_chipseq_samples_library_ids.csv|cut -f 1|\
    awk -v lib=$b -v path=$c -v mark=$a -F "-01" '{print $1"\t"lib"\t"mark"\t"path}' ;done \
    < d.tmp |sort -k1 -k2 > $mark"_peak_file_path.txt"

    rm a.tmp  d.tmp
    head -2 $mark"_peak_file_path.txt"
    wc -l $mark"_peak_file_path.txt";
done

/projects/da_workspace/software/ChromHMM/Cervical_all_patients,
HTMCP-03-06-02001	A84615	H3K27ac	/projects/chipseq/macs/A84615/75nt/hg19a/bwa-mem-0.7.6a-sb/MACS_2.1.1.20160309/370/A84615_H3K27ac_peaks.narrowPeak
HTMCP-03-06-02002	A94516	H3K27ac	/projects/chipseq/macs/A94516/75nt/hg19a/bwa-mem-0.7.6a-sb/MACS_2.1.1.20160309/691/A94516_H3K27ac_peaks.narrowPeak
54 H3K27ac_peak_file_path.txt
HTMCP-03-06-02001	A84613	H3K27me3	/projects/chipseq/macs/A84613/75nt/hg19a/bwa-mem-0.7.6a-sb/MACS_2.1.1.20160309/374/A84613_H3K27me3_peaks.broadPeak
HTMCP-03-06-02002	A94514	H3K27me3	/projects/chipseq/macs/A94514/75nt/hg19a/bwa-mem-0.7.6a-sb/MACS_2.1.1.20160309/689/A94514_H3K27me3_peaks.broadPeak
54 H3K27me3_peak_file_path.txt
HTMCP-03-06-02001	A84614	H3K36me3	/projects/chipseq/macs/A84614/75nt/hg19a/bwa-mem-0.7.6a-sb/MACS_2.1.1.20160309/371/A84614_H3K36me3_peaks.broadPeak
HTMCP-03-06-02002	A94515	H3K36me3	/projects/chipseq/macs/A94515/75nt/hg19a/bwa-mem-0.7.6a-sb/MACS_2.1.1.20160309/690/A94515_H3K36me3

In [ ]:
# format and concatenate the peak files from each mark

### get new h3k36me3 new peak file

In [3]:
f1 = '/projects/da_workspace/software/ChromHMM/Cervical_all_patients/H3K36me3_peak_file_path.txt.with.6.marks'
f2 = '/projects/da_workspace/software/ChromHMM/cervical_all_patients_new_h3k36me3/h3k36me3_new_files.txt'
df1 = pd.read_csv(f1, sep='\t', header=None)

df2 = pd.read_csv(f2, sep='\t', header=None)
df1.head(2)
df2.head(2)

,0,1,2,3
0,HTMCP-03-06-02001,A84614,H3K36me3,/projects/chipseq/macs/A84614/75nt/hg19a/bwa-m...
1,HTMCP-03-06-02002,A94515,H3K36me3,/projects/chipseq/macs/A94515/75nt/hg19a/bwa-m...


,0,1,2
0,B07688,A84735,/projects/chipseq/macs/B07688/75nt/hg19a/bwa-m...
1,B07684,A84644,/projects/chipseq/macs/B07684/75nt/hg19a/bwa-m...


In [25]:
f3 = '/projects/da_workspace/software/ChromHMM/cervical_all_patients_new_h3k36me3/HTMCP_CaCx_ChIPseq_IDs_final_for_submission_April29_2019_v2.xls'
df3 = pd.read_excel(f3)
df3 = df3[['Biospecimen ID', 'H3K36me3 library ID']]
df3['Biospecimen ID'] = df3['Biospecimen ID'].apply(lambda x: '-'.join(x.split('-')[:-1]))
df3.head()
df3.shape

,Biospecimen ID,H3K36me3 library ID
0,HTMCP-03-06-02063,B07616
1,HTMCP-03-06-02214,B07622
2,HTMCP-03-06-02042,B07614
3,HTMCP-03-06-02332,B07623
4,HTMCP-03-06-02185,B07620


(52, 2)

In [26]:
df4 = pd.merge(df3, df2, left_on='H3K36me3 library ID', right_on=0, how='left').drop([0,1], axis=1)

In [28]:
df4.head(2)
df4.shape

,Biospecimen ID,H3K36me3 library ID,2
0,HTMCP-03-06-02063,B07616,/projects/chipseq/macs/B07616/75nt/hg19a/bwa-m...
1,HTMCP-03-06-02214,B07622,/projects/chipseq/macs/B07622/75nt/hg19a/bwa-m...


(52, 3)

In [37]:
df5 = pd.merge(df1, df4, left_on=0, right_on='Biospecimen ID', how='left').drop([1, 3,'Biospecimen ID'], axis=1)
df5.columns = range(4)

In [38]:
df5[[0,2,1,3]].shape

(47, 4)

In [39]:
df5 = df5.fillna('H3K36me3')

In [40]:
of ='/projects/da_workspace/software/ChromHMM/cervical_all_patients_new_h3k36me3/H3K36me3_peak_file_path.txt.with.6.marks'
df5.to_csv(of, index=False, header=False, sep='\t')

In [ ]:
# rerun on 20190429

In [2]:
%%bash
ssh gphost06
# step 1: make chromhmm input file cellmarkerfiletable.txt, this file needs to be 3 columns, patient, mark, and mark peak file.
# step 2: create symlink to the MCS2 broadPeaks and narrowPeaks files
cd /projects/da_workspace/software/ChromHMM/cervical_all_patients_new_h3k36me3
cat *.marks|awk -F "/" '{print $0"\t"$11}' > cellmarkfiletable.txt.full.path
while read a b c d;do grep $a cellmarkfiletable.txt.full.path;done < H3K9me3_peak_file_path.txt.with.6.marks>cellmarkfiletable.txt.full.path.6.marks
mkdir /projects/da_workspace/software/ChromHMM/cervical_all_patients_new_h3k36me3/peak_bed_files
cd /projects/da_workspace/software/ChromHMM/cervical_all_patients_new_h3k36me3/peak_bed_files
while read a b c d e;do ln -s $d $e;done < /projects/da_workspace/software/ChromHMM/cervical_all_patients_new_h3k36me3/cellmarkfiletable.txt.full.path.6.marks

Pseudo-terminal will not be allocated because stdin is not a terminal.


In [3]:
%%bash
cd /projects/da_workspace/software/ChromHMM/cervical_all_patients_new_h3k36me3
cat cellmarkfiletable.txt.full.path.6.marks|cut -f 1,3,5 > cellmarkfiletable.txt

In [4]:
%%bash
ssh gphost06
# run chromHMM BinarizeBed, memory may need to significantly larger, e.g. -Xmx120G
cd /projects/da_workspace/software/ChromHMM
/gsc/software/linux-x86_64-centos6/jdk1.8.0_102/bin/java -Xmx120G -jar ChromHMM.jar BinarizeBed -b 200 -peaks CHROMSIZES/hg19.txt /projects/da_workspace/software/ChromHMM/cervical_all_patients_new_h3k36me3/peak_bed_files/ /projects/da_workspace/software/ChromHMM/cervical_all_patients_new_h3k36me3/cellmarkfiletable.txt /projects/da_workspace/software/ChromHMM/cervical_all_patients_new_h3k36me3/binary/

Writing to file /projects/da_workspace/software/ChromHMM/cervical_all_patients_new_h3k36me3/binary//HTMCP-03-06-02012_1_binary.txt
Writing to file /projects/da_workspace/software/ChromHMM/cervical_all_patients_new_h3k36me3/binary//HTMCP-03-06-02012_2_binary.txt
Writing to file /projects/da_workspace/software/ChromHMM/cervical_all_patients_new_h3k36me3/binary//HTMCP-03-06-02012_3_binary.txt
Writing to file /projects/da_workspace/software/ChromHMM/cervical_all_patients_new_h3k36me3/binary//HTMCP-03-06-02012_4_binary.txt
Writing to file /projects/da_workspace/software/ChromHMM/cervical_all_patients_new_h3k36me3/binary//HTMCP-03-06-02012_5_binary.txt
Writing to file /projects/da_workspace/software/ChromHMM/cervical_all_patients_new_h3k36me3/binary//HTMCP-03-06-02012_6_binary.txt
Writing to file /projects/da_workspace/software/ChromHMM/cervical_all_patients_new_h3k36me3/binary//HTMCP-03-06-02012_7_binary.txt
Writing to file /projects/da_workspace/software/ChromHMM/cervical_all_patients_new_

Pseudo-terminal will not be allocated because stdin is not a terminal.


In [5]:
%%bash
# add chr suffix to binary files
ssh gphost06
cd /projects/da_workspace/software/ChromHMM/cervical_all_patients_new_h3k36me3/binary/
for f in *_binary.txt;do awk '/^HTMCP/{$2="chr"$2} 1' $f |sed 's/ /\t/g'> $f.chr_suffix;done 
rm *_binary.txt

# run chromHMM, need to run on gphost because of memory requirement
# /gsc/software/linux-x86_64-centos6/jdk1.8.0_102/bin/java -Xmx50g -jar ChromHMM.jar LearnModel -b 200 -p 0 ./Cervical_all_patients/binary/ ./Cervical_all_patients/18_states 18 hg19

# or run for multiple states models
cd /projects/da_workspace/software/ChromHMM
for f in 15 16 17 18 19 20 21;do mkdir ./cervical_all_patients_new_h3k36me3/$f"_states"; /gsc/software/linux-x86_64-centos6/jdk1.8.0_102/bin/java -Xmx50g -jar ChromHMM.jar LearnModel -b 200 -p 0 ./cervical_all_patients_new_h3k36me3/binary/ ./cervical_all_patients_new_h3k36me3/$f"_states" $f hg19;done
for f in 22 23 24;do /gsc/software/linux-x86_64-centos6/jdk1.8.0_102/bin/java -Xmx100g -jar ChromHMM.jar LearnModel -b 200 -p 0 ./cervical_all_patients_new_h3k36me3/binary/ ./cervical_all_patients_new_h3k36me3/$f"_states" $f hg19;done

Using 144 threads for Baum-Welch training
Writing to file ./cervical_all_patients_new_h3k36me3/15_states/transitions_15.txt
Writing to file ./cervical_all_patients_new_h3k36me3/15_states/emissions_15.txt
Writing to file ./cervical_all_patients_new_h3k36me3/15_states/emissions_15.svg
Writing to file ./cervical_all_patients_new_h3k36me3/15_states/emissions_15.png
Writing to file ./cervical_all_patients_new_h3k36me3/15_states/transitions_15.png
Writing to file ./cervical_all_patients_new_h3k36me3/15_states/transitions_15.svg
Writing to file ./cervical_all_patients_new_h3k36me3/15_states/model_15.txt
 Iteration  Estimated Log Likelihood     Change    Total Time (secs)
         1            -426484288.820          -                 65.3
         2            -343189220.859 83295067.961                103.3
         3            -335150258.385 8038962.474                142.2
         4            -332550425.834 2599832.551                179.4
         5            -330115146.151 2435279.68

Pseudo-terminal will not be allocated because stdin is not a terminal.
